In [1]:
# sample of training the titanic data sets and creating a submission using AutoML 
# for the popular Kaggle Titanic challenge. Note: this has no feature engineering. 

In [2]:
# Uncomment below to Install azureml.core in the current Jupyter kernel
# import sys
# !{sys.executable} -m  pip install azureml.core --user

In [3]:
# Uncomment below to Install azureml.train.automl in the current Jupyter kernel
# import sys
# !{sys.executable} -m  pip install azureml.train.automl --user

In [4]:
subscription_id = ""
resource_group = "auto-ml-local"
workspace_name = "kappgle-ml-local"
workspace_region = "eastus2"

In [5]:
# Import the Workspace class and check the Azure ML SDK version.
from azureml.core import Workspace

ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,
                      exist_ok=True)
ws.get_details()

{'applicationInsights': '/subscriptions/c8a23972-1b42-43fa-9bda-92e665014f30/resourcegroups/auto-ml-local/providers/microsoft.insights/components/kappgleminsightsswmwdewl',
 'containerRegistry': '/subscriptions/c8a23972-1b42-43fa-9bda-92e665014f30/resourcegroups/auto-ml-local/providers/microsoft.containerregistry/registries/kappglemacrvuatnpjh',
 'creationTime': '2019-02-11T00:06:12.4721271+00:00',
 'description': '',
 'friendlyName': 'kappgle-ml-local',
 'id': '/subscriptions/c8a23972-1b42-43fa-9bda-92e665014f30/resourceGroups/auto-ml-local/providers/Microsoft.MachineLearningServices/workspaces/kappgle-ml-local',
 'identityPrincipalId': 'c037cc6f-9b6d-4abb-b237-0927801f8514',
 'identityTenantId': '72f988bf-86f1-41af-91ab-2d7cd011db47',
 'identityType': 'SystemAssigned',
 'keyVault': '/subscriptions/c8a23972-1b42-43fa-9bda-92e665014f30/resourcegroups/auto-ml-local/providers/microsoft.keyvault/vaults/kappglemkeyvaultuirqpheu',
 'location': 'eastus2',
 'name': 'kappgle-ml-local',
 'stora

In [6]:
import azureml.core
import pandas as pd
from azureml.core.workspace import Workspace
import logging
import os

# ws = Workspace.from_config()
# choose a name for the run history container in the workspace
experiment_name = 'automated-ml-classification'
# project folder
project_folder = './automated-ml-classification'

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(data=output, index=['']).T

,
Location,eastus2
Project Directory,./automated-ml-classification
Resource Group,auto-ml-local
SDK version,1.0.10
Subscription ID,c8a23972-1b42-43fa-9bda-92e665014f30
Workspace,kappgle-ml-local


In [7]:
import pickle
import sys
import os
import pandas
import numpy as np
from sklearn import datasets
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve

#get the titanic training data set
url = "https://raw.githubusercontent.com/TheovanKraay/mldata/master/titanic-train.csv"
titanic = pandas.read_csv(url)
#dflow_X = titanic.drop('Survived', axis=1).values 
dflow_X = titanic[['Sex', 'Age', 'SibSp', 'Parch', 'Cabin', 'Embarked', 'Fare']]
dflow_y = titanic['Survived']

In [8]:
from sklearn.model_selection import train_test_split

#split the dataset
x_df = dflow_X
y_df = dflow_y
x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.2, random_state=223)

In [9]:
#configure AutoML settings
automl_settings = {
    "iteration_timeout_minutes" : 10,
    "iterations" : 30,
    "primary_metric" : 'AUC_weighted',
    "preprocess" : True,
    "verbosity" : logging.INFO,
    "n_cross_validations": 5
}

In [10]:
from azureml.train.automl import AutoMLConfig

# local compute - set the parameters
automated_ml_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automated_ml_errors.log',
                             path = project_folder,
                             X = x_train.values,
                             y = y_train.values.flatten(),
                             **automl_settings)

C:\Users\thvankra\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [11]:
#run the training

from azureml.core.experiment import Experiment
experiment=Experiment(ws, experiment_name)
local_run = experiment.submit(automated_ml_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_488ab5ac-d399-4cab-b651-4b1e50a0d0ee
*******************************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
TRAINFRAC: Fraction of the training data to train on.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
*******************************************************************************************************************

 ITERATION   PIPELINE                                       TRAINFRAC  DURATION      METRIC      BEST
         0   MaxAbsScaler LightGBM                          1.0000     0:00:16       0.8308    0.8308
         1   MaxAbsScaler LightGBM                          1.0000     0:00:14       0.8310    0.8310
         2   StandardScalerWrapper LightGBM                 1.0000

In [12]:
#get the best fitted model
best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: automated-ml-classification,
Id: AutoML_488ab5ac-d399-4cab-b651-4b1e50a0d0ee_29,
Type: None,
Status: Completed)
Pipeline(memory=None,
     steps=[('datatransformer', DataTransformer(logger=None, task=None)), ('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(classification_labels=None,
               estimators=[('LinearSVM', Pipeline(memory=None,
     steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('linearsvmwrapper', ...666666666667, 0.13333333333333333, 0.06666666666666667, 0.13333333333333333, 0.26666666666666666]))])


In [13]:
#predict against the Kaggle provided test data for the Titanic
url = "https://raw.githubusercontent.com/TheovanKraay/mldata/master/kaggle-test.csv"
test = pandas.read_csv(url)

testpred = test[['Sex', 'Age', 'SibSp', 'Parch', 'Cabin', 'Embarked', 'Fare']]
y_predict = fitted_model.predict(testpred.values)

testpred = test[['PassengerId','Sex', 'Age', 'SibSp', 'Parch', 'Cabin', 'Embarked', 'Fare']]

In [14]:
#generate a submission
sub = pandas.DataFrame()
sub['PassengerId'] = test['PassengerId']
sub['Survived'] = y_predict
#print(sub)

In [15]:
#write the submission file
sub.to_csv('submission.csv',index=False)